https://www.kaggle.com/code/daisypond/example-of-topic-modeling-for-tweets

https://www.kaggle.com/code/errearanhas/topic-modelling-lda-on-elon-tweets

## Importing libs

In [1]:
import pandas as pd
import os
from tqdm import tqdm 
import numpy as np
import pickle
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Importing data

In [2]:
df = (
    pd.read_csv('tweets/all_tweets.csv')
    .iloc[:, 1:]
)

In [3]:
df.shape

(359036, 28)

In [4]:
tweets = df['text'].drop_duplicates()

In [5]:
tweets.shape

(304154,)

In [6]:
import re
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk

In [7]:
tweets.head()

0    Kararlarındaki temel dayanak, hukukun evrensel...
1    #OhalKomisyonuHukuksuzdur https://t.co/3aUr5MZWPU
2    RT @Yozgat_KHK: #OhalKomisyonuHukuksuzdur\nÇün...
3    #OhalKomisyonuHukuksuzdur https://t.co/FHdc5DQ0u0
4    RT @Malatya_KHK: Öldükten sonra insanlari işe ...
Name: text, dtype: object

In [15]:
tweets = tweets.map(lambda x: re.sub('#[A-Za-z0-9]+', '', x))  # remove hashtag
tweets = tweets.map(lambda x: re.sub('https://t.co/[A-Za-z0-9]+', '', x)) # remove link
tweets = tweets.map(lambda x: re.sub('[,\.!?]', '', x)) # remove ,.!?
tweets = tweets.map(lambda x: re.sub('\n', '', x)) # remove \n
tweets = tweets.map(lambda x: unidecode(x)) # remove accents

stopwords = nltk.corpus.stopwords.words('turkish') 
stopwords = [unidecode(i) for i in stopwords] 
tweets = tweets.apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)])) # remove stopwords

In [16]:
tweets = tweets.str.lower() # lower all the words

In [19]:
tweets = tweets.drop_duplicates()

In [ ]:
text_len = tweets.apply(lambda x: len(x))

In [30]:
tweets = tweets[text_len[text_len>5].index] # selecting only tweets that are greater than 5 chars

In [31]:
# bow
vect = CountVectorizer(max_features=10000)
x = vect.fit_transform(tweets)

In [32]:
topics = 5

In [33]:
# lda
lda = LatentDirichletAllocation(n_components = topics, learning_method="batch", max_iter=20, random_state=42)
document_topics = lda.fit_transform(x)

In [55]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print('\n')

In [52]:
feat_names = vect.get_feature_names()

In [56]:
display_topics(lda, feat_names, 20)

Topic 0:
rt khk turkiye_khk hukuk li adalet bir ohal hak ankara_khk lilar olarak samsun_khk lar ihrac insan mkorkmaz62 herkesicinchp ceza istiyoruz


Topic 1:
bir rt degil yok var kadar olan bile olarak tek insan sadece artik ben zaman adalet boyle buyuk gun hukuk


Topic 2:
rt bir in cezaevinde olsun tutuklu hasta olan adalet_bakanlik tahliye yil once yusuf cezaevi adalet cocuk yasindaki gergerliogluof yildir gun


Topic 3:
rt nin da de in turkiye bir kadin karsi istanbul haklari olarak bugun ye tarafindan baskani insan genel kadinlar li


Topic 4:
the rt of in to is and for on be that are we with has this it turkey by not


